In [5]:
import os
import pandas as pd
import numpy as np
import statsmodels.api as sm
import warnings
from statsmodels.tsa.arima.model import ARIMA
from scipy.spatial.distance import pdist, squareform
from sklearn.cluster import AgglomerativeClustering
from joblib import Parallel, delayed
from scipy.stats import norm
import time
from itertools import product
from tqdm import tqdm

# Function to determine the best ARIMA order and keep a rolling model copy
# Intermediate function
def best_arima(series, max_p=5, max_q=5, max_d=3):
    """
    Automatically selects the best ARIMA hyperparameters (p, d, q) based on AIC and fits the model.
    Returns both the best order and the fitted model (model_fit).
    If no valid model is found, returns (None, None).
    """
    def _try_arima(p, d, q, series):
        try:
            # Suppress warnings during model fitting
            with warnings.catch_warnings():
                warnings.simplefilter("ignore")
                model = ARIMA(series, order=(p, d, q))
                model_fit = model.fit()
            return (p, d, q), model_fit, model_fit.aic
        except:
            return None, None, float('inf')
    
    # Parallel processing of ARIMA models
    param_combinations = list(product(range(1,max_p), range(1, max_d), range(1, max_q)))
    results = Parallel(n_jobs=-1)(
        delayed(_try_arima)(p, d, q, series) 
        for p, d, q in param_combinations
    )
    
    # Find the best model
    best_aic = float('inf')
    best_order = None
    best_model_fit = None
    
    for order, model_fit, aic in results:
        if aic < best_aic and order is not None:
            best_aic = aic
            best_order = order
            best_model_fit = model_fit
    
    if best_order is None:
        print("Warning: No valid ARIMA model found for the series")
        
    return best_order, best_model_fit



def Forecast(series, train_l=400, h_lst=[1, 3, 6, 12]):
    """
    Forecasts using ARIMA with a rolling window.
    - series: the time series data
    - train_l: length of the training set (400).
    - out_l: length of the out-of-sample data (R).
    - h: forecast horizon.
    - max_p, max_q: maximum order for ARIMA parameters (p, q). 
    """

    h_max = max(h_lst)
    forecasts = []
    residuals = []
    out_l = len(series) - train_l
    print(series.shape)  
    print(out_l)

    for i in range(out_l - 1):  # Loop over each time step in the out-of-sample period
        # Define the training window
        train_data = series[i:i+train_l]
        
        # Fit ARIMA model on the training window
        best_order, best_model_fit = best_arima(train_data)  # ARIMA(p, d, q)
        location = np.array(h_lst) - 1
        forecast = best_model_fit.forecast(steps=h_max)[location]
        print(forecast)
        print()
        forecasts.append(forecast)
        
    return forecasts

# colloect forecasts and residuals for all series for some horizon h

def collect_forecasts(S, Basey, train_l, h_lst=[1,3,6,12]):
    
    '''
    # Collect forecasts and residuals for all series for horizon h
    # Parameters:
    # S: summation matrix, (num_of_series, num_of_base_stock)
    # Basey: compositions of index, (num_of_dates, num_of_base_stock) 
    # train_l: length of training window, default=400
    # out_l: length of out-of-sample period
    # h: forecast horizon    
    '''

    Y_mat = S @ np.array(Basey).T # (num of series, num of dates)
    forecasts_all = []
    
    for i in tqdm(range(Y_mat.shape[0]), desc="Processing series"):
        series = Y_mat[i]
        forecasts = Forecast(series, train_l, h_lst)
        forecasts_all.append(forecasts)
    
    # Save forecasts to a CSV file
    forecasts_df = pd.DataFrame(forecasts_all)
    forecasts_df.to_csv('data/forecasts.csv', index=False)
    return np.array(forecasts_all)

def G_mat(S, forecasts_all, residuals_all):

    """
    Perform MinT reconciliation using the summation matrix S and the covariance matrix W_h.
    residuals_all: matrix of residuals, (num of series, out_l)
    S: summation matrix, (n_a + num of stock, num of stock)
    forecasts_all: matrix of forecasts, (num of series, out_l)
    """
    S = np.array(S)
    row, column = forecasts_all.shape[0], forecasts_all.shape[1] #

    # calculate covariance matrix W_h
    W_h = np.cov(residuals_all) # (num of series, num of series)
    print(W_h.shape)

    # check if the covariance matrix is correct
    if W_h.shape[0] != row:
        print("Fail to calculte the covariance matrix")
        return None
    
    # calculate MinT reconciliation matrix G_h, (num of base stock, num of series)
    W_inverse = np.linalg.inv(W_h)

    # Print shapes for debugging
    print("S.T shape:", S.T.shape)
    print("W_inverse shape:", W_inverse.shape)
    print("S shape:", S.shape)

    # Check the data types of the matrices
    print("Data type of S:", type(S))
    print("Data type of W_inverse:", type(W_inverse))

    

    try:
        G_h = np.linalg.inv(S.T @ W_inverse @ S) @ (S.T @ W_inverse)
    except ValueError as e:
        print("Matrix alignment error:", e)
        return None

    G_h = np.linalg.inv(S.T @ W_inverse @ S) @ S.T @ W_inverse

    return G_h

In [6]:
# Load data
Basey = pd.read_excel(os.path.join("data", "DataP0.xlsx"), index_col='Dates0')
# DJIA_dat = pd.read_excel(os.path.join("data", "DJIA - composition at 2020.xlsx"))   
h_lst = [1,3,6,12]
S = np.array(pd.concat([pd.DataFrame(np.ones((1, Basey.shape[1]))/0.15), pd.DataFrame(np.eye(Basey.shape[1]))], axis=0)) # (Nnum of base stock+1,num of base stock)

In [7]:
forecasts_all = collect_forecasts(S, Basey, 400, h_lst) #(num_series, num_forecasts, num_horizons)

Processing series:   0%|          | 0/31 [00:00<?, ?it/s]

(524,)
124
[232571.67774421 232569.28750595 232585.37456067 232563.74241453]

[233054.05734883 233000.08689227 233080.79588331 232996.22821974]

[231453.46022424 231624.27265339 232090.40551896 232717.15269404]

[230242.07652807 230414.54328672 230828.07677315 231327.48681735]

[230723.22913084 230650.07431039 230859.93727636 230685.98352477]

[231683.53365271 231726.18903745 231802.60638703 231732.00888899]

[228916.31986877 228900.70778133 228897.54941719 228897.29063832]

[228349.19342609 228429.26962863 228347.73666276 228467.86974016]

[230665.99580757 230665.58497926 230665.58317525 230665.58317578]

[230007.29167494 230000.06702616 229951.82628571 229996.19456964]

[229695.17076836 229669.38106048 229474.07020143 229658.33584662]

[233050.29372625 233010.83900755 233099.80394765 233067.88364417]

[234673.48554293 234921.79544137 235393.72716993 236107.70323006]

[232154.83143594 232162.80671266 232171.03616833 232169.32376635]

[225839.95721321 225848.01821312 225859.90673347 22

Processing series:   3%|▎         | 1/31 [10:03<5:01:42, 603.42s/it]

[196348.00084626 196350.26427914 196350.41859185 196350.41619614]

(524,)
124
[1157.31848681 1161.98004918 1165.64820134 1167.73622277]

[1187.82426991 1189.44475813 1191.07532698 1195.9435521 ]

[1165.76724305 1166.92839885 1170.42306709 1175.22713103]

[1143.141092   1144.87814867 1147.48373645 1152.69491201]

[1145.34895081 1147.3634195  1150.38479568 1156.42754795]

[1131.28576138 1133.06965781 1135.75555109 1141.12734371]

[-31448.96394072 -31610.49309163 -31923.74291981 -32407.01313969]

[1115.74464821 1117.64329508 1120.48294005 1126.16222487]

[1134.31690703 1135.67831881 1138.57903872 1144.3226313 ]

[1099.0155328  1100.68755129 1103.2122383  1108.26161927]

[1099.86433189 1100.15154669 1102.98889797 1108.67883425]

[1114.79580172 1115.86811329 1118.87801444 1124.79339461]

[1106.59065692 1113.77231303 1107.69187266 1113.10709605]

[1108.02583884 1109.07610737 1111.90107286 1117.68595301]

[1076.97258221 1078.26713628 1080.97020466 1086.25380135]

[1081.61678327 1085.38799291 

Processing series:   6%|▋         | 2/31 [23:21<5:47:01, 717.99s/it]

...

[254.87326721 254.84165405 254.83939912 254.8393587 ]

(524,)
124
[285.00037368 284.77888343 284.76006359 284.75962826]

[282.27194903 282.13772368 282.1247629  282.12440172]

[283.16033706 283.14459386 283.14312887 283.14309018]

[288.87040503 288.98776271 288.99743908 288.99765331]

[294.9456856  295.48651241 295.52332302 295.54920545]

[296.33854714 296.36680551 296.30578004 296.26253868]

[288.96869843 288.8499973  288.84119221 288.84102512]

[287.86778718 287.81700222 287.81328013 287.81321072]

[290.54820847 290.58581977 290.58835377 290.58839556]

[291.90306553 291.93630905 291.93850837 291.93854368]

[289.6817431  289.650033   289.64777258 289.6477321 ]

[297.33825491 297.4809049  297.49284036 297.49311031]

[301.7582371  299.04041079 294.38312301 292.94509071]

[305.0243222  305.09421936 305.0996284  305.09973758]

[296.09643972 295.95773718 295.94899975 295.94886914]

[295.42935444 295.38428239 295.38134139 295.38129511]

[291.95511947 291.87988636 291.87465136 291.87456087]


Processing series:  97%|█████████▋| 30/31 [5:05:33<11:13, 673.94s/it]

[215.59562003 215.55728852 214.38578537 213.54829785]

(524,)
124
[1000.15419324 1004.98079951 1000.94680882 1004.75820169]

[ 998.39842743  999.6054321  1000.14120511 1000.78637792]

[1002.88469432 1002.75760289  998.21259636  999.16619944]

[1027.1268061  1022.63440285 1021.25259543 1020.34727233]

[1032.16186866 1029.54122985 1032.23093068 1028.45329122]

[1039.33752367 1036.99221741 1041.14221544 1038.51680253]

[1017.96232828 1017.32224261 1022.74702836 1020.35858179]

[1012.03778719 1014.68405303 1017.15549699 1017.40094458]

[1039.77537038 1047.11267234 1043.38754472 1047.60594357]

[1043.5168141  1045.6010476  1037.76992434 1042.6612662 ]

[177234.60759725 178123.82662271 179003.22805315 181633.98070691]

[1044.27925747 1039.68180997 1038.16812562 1037.03326208]

[1053.35050191 1049.17364627 1053.26774509 1049.18103929]

[1054.08016691 1051.30141272 1057.45649677 1053.79288185]

[1034.84198534 1035.74420548 1041.48975735 1039.19178968]

[1037.88637106 1041.62008224 1042.9419102

Processing series: 100%|██████████| 31/31 [5:19:17<00:00, 617.97s/it]

[873.4197717  874.76638676 875.29522021 871.99907539]



In [8]:
np.save('data/forecasts_2020.npy', np.array(forecasts_all))

In [69]:
G_lst = []
train_l = 400
total_l = len(Basey)
for i in range(len(h_lst)):
    forecast_mat = forecasts_all[:,:,i] # (num of series, num of forecasts), (31, 123)
    # print(forecast_mat.shape)

    Y_mat = np.array(S @ np.array(Basey).T) # (num of series, num of dates)

    h = h_lst[i] # [1, 3, 6, 12]
    forecast_range = np.array(range(h+train_l-1, total_l-1)) 
    # print(num_of_forecasts)
    num_of_forecasts = len(forecast_range)

    # calculate residuals
    residuals_mat = forecast_mat[:, 0:num_of_forecasts] - Y_mat[:, forecast_range] # (num of series, num of forecasts)
    # print(residuals_mat.shape)

    G_h = G_mat(S, forecast_mat, residuals_mat) # (num of base stock, num of series)
    G_lst.append(G_h)

G_lst = np.array(G_lst)
print(G_lst.shape)
np.save('data/G_lst.npy', G_lst)



(31, 31)
S.T shape: (30, 31)
W_inverse shape: (31, 31)
S shape: (31, 30)
Data type of S: <class 'numpy.ndarray'>
Data type of W_inverse: <class 'numpy.ndarray'>
(31, 31)
S.T shape: (30, 31)
W_inverse shape: (31, 31)
S shape: (31, 30)
Data type of S: <class 'numpy.ndarray'>
Data type of W_inverse: <class 'numpy.ndarray'>
(31, 31)
S.T shape: (30, 31)
W_inverse shape: (31, 31)
S shape: (31, 30)
Data type of S: <class 'numpy.ndarray'>
Data type of W_inverse: <class 'numpy.ndarray'>
(31, 31)
S.T shape: (30, 31)
W_inverse shape: (31, 31)
S shape: (31, 30)
Data type of S: <class 'numpy.ndarray'>
Data type of W_inverse: <class 'numpy.ndarray'>
(4, 30, 31)
